In [ ]:
!pip install -q chromadb sentence-transformers --quiet

In [7]:
import chromadb
from chromadb.config import Settings
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from sentence_transformers import SentenceTransformer
from datetime import datetime
import time
import os

In [8]:
from os.path import exists
persist_dir = "./user_memory_db"
if not os.path.exists(persist_dir):
  os.makedirs(persist_dir, exist_ok=True)

In [9]:
embedding_fn = SentenceTransformerEmbeddingFunction(model_name = "all-MiniLM-L6-v2")
client= chromadb.PersistentClient(path = persist_dir)

In [10]:
collection = client.get_or_create_collection(name="multi_user_memory",embedding_function=embedding_fn)

In [12]:
def store_message(user_id,message):
  timestamp = time.time()
  collection.add(documents=[message], metadatas=[{ "user_id": user_id, "timestamp": timestamp }],
                 ids=[f"{user_id}_{str(timestamp)}"]
                 )
  print("Agent: Noted.")

In [13]:
# Function to retrieve messages only for current user
def show_user_history(user_id):
    results = collection.query(
        query_texts=["user memory"],
        n_results=10,
        where={"user_id": user_id}
    )
    print(f"Last 10 messages for {user_id}:")
    for doc in results['documents'][0]:
        print(f" - {doc}")

In [14]:
# Function to clear memory for a specific user
def clear_user_history(user_id):
    collection.delete(
        where={"user_id": user_id}
    )
    print(f"Memory cleared for {user_id}")

In [ ]:
# Simulated multi-user interaction loop
print("Multi-User Memory Tracker (type 'show', 'clear', 'logout', or 'exit')\n")
exit_flag = False
while not exit_flag:
    user_id = input("Enter your user ID (or type 'exit' to quit): ").strip()
    if user_id.lower() == "exit":
        print("Exiting program.")
        break  # Exit from outer loop
    print(f"\nLogged in as: {user_id}. Start chatting below.")
    while True:
        message = input(f"{user_id} > ").strip()
        if message.lower() == "logout":
            print(f"Logged out {user_id}.\n")
            break  # Return to user ID prompt
        elif message.lower() == "exit":
            print("Exiting program.")
            exit_flag = True
            break  # Break inner loop, then outer will see flag and stop
        elif message.lower() == "show":
            show_user_history(user_id)
        elif message.lower() == "clear":
            clear_user_history(user_id)
        else:
            store_message(user_id, message)